# date_time

In [ ]:
open rust.rust_operators
open sm'_operators

In [ ]:
//// test

open testing

## date_time

### timestamp

In [ ]:
nominal timestamp_gleam =
    `(
        backend_switch `(()) `({}) {
            Gleam = (fun () => global "import gleam/time/timestamp") : () -> ()
        }
        $'' : $'timestamp.Timestamp'
    )
nominal timestamp_python =
    `(
        backend_switch `(()) `({}) {
            Python = (fun () => global "import datetime") : () -> ()
        }
        $'' : i64
    )
type timestamp_switch =
    {
        Gleam : timestamp_gleam
        Fsharp : i64
        Python : timestamp_python
    }
nominal timestamp = $'backend_switch `(timestamp_switch)'

### timestamp_guid

In [ ]:
type timestamp_guid = guid.guid

### date_time_guid

In [ ]:
type date_time_guid = guid.guid

### test_guid

In [ ]:
//// test

inl test_guid () =
    "6543210F-EDCB-A987-6543-210FEDCBA987" |> guid.new_guid

## day_of_week

In [ ]:
union day_of_week =
    | Sunday
    | Monday
    | Tuesday
    | Wednesday
    | Thursday
    | Friday
    | Saturday

## month

In [ ]:
union month =
    | January
    | February
    | March
    | April
    | May
    | June
    | July
    | August
    | September
    | October
    | November
    | December

## day

In [ ]:
nominal day = int

## year

In [ ]:
nominal year = int

## fsharp

### date_time

In [ ]:
nominal date_gleam =
    `(
        backend_switch `(()) `({}) {
            Gleam = (fun () =>
                global "import gleam/time/calendar"
                        ) : () -> ()
        }
        $'' : $'calendar.Date'
    )
nominal time_of_day_gleam =
    `(
        backend_switch `(()) `({}) {
            Gleam = (fun () =>
                global "import gleam/time/calendar"
                        ) : () -> ()
        }
        $'' : $'calendar.TimeOfDay'
    )
nominal duration_gleam =
    `(
        backend_switch `(()) `({}) {
            Gleam = (fun () =>
                global "import gleam/time/duration"
                        ) : () -> ()
        }
        $'' : $'duration.Duration'
    )

nominal date_time_python =
    `(
        backend_switch `(()) `({}) {
            Python = (fun () => global "import datetime") : () -> ()
        }
        $'' : $'datetime.datetime'
    )
type date_time_switch =
    {
        Gleam : date_gleam * time_of_day_gleam * duration_gleam
        Fsharp : $'System.DateTime'
        Python : date_time_python
    }
nominal date_time = $'backend_switch `(date_time_switch)'

### calendar_month

In [ ]:
nominal calendar_month =
    `(
        backend_switch `(()) `({}) {
            Gleam = (fun () =>
                global "import gleam/time/calendar"
                        ) : () -> ()
        }
        $'' : $'calendar.Month'
    )

### tempo_date_time

In [ ]:
nominal tempo_date_time =
    `(
        backend_switch `(()) `({}) {
            Gleam = (fun () =>
                global "import tempo"
                        ) : () -> ()
        }
        $'' : $'tempo.DateTime'
    )

### year

In [ ]:
inl get_year (date_time : date_time) : i32 =
    backend_switch {
        Gleam = fun () => $'0' : i32
        Fsharp = fun () => date_time |> $'_.Year' : i32
        Python = fun () => $'!date_time.year' : i32
    }

### format

In [ ]:
inl format (format : string) (date_time : date_time) : string =
    backend_switch {
        Gleam = fun () =>
            global "import tempo/datetime"
            inl date : date_gleam = $'!date_time.0'
            inl time_of_day : time_of_day_gleam = $'!date_time.1'
            inl duration : duration_gleam = $'!date_time.2'
            inl timestamp : timestamp = $'timestamp.from_calendar(!date, !time_of_day, !duration)'
            inl tempo_date_time : tempo_date_time = $'!timestamp |> datetime.from_timestamp'
            inl tempo_date_time : tempo_date_time = $'!tempo_date_time |> datetime.add(!duration)'
            inl format =
                if format = ""
                then "M-D-YY hh:mm:ss A"
                else format
            $'!tempo_date_time |> datetime.format(tempo.Custom(!format))' : string
            // { date_time format date time_of_day duration timestamp tempo_date_time } |> sm'.format
        Fsharp = fun () =>
            inl format =
                if format = ""
                then "M-d-y hh:mm:ss tt"
                else format
            $'!date_time.ToString' format : string
        Python = fun () =>
            inl date_time = join date_time
            if format <> ""
            then $'!date_time.strftime(!format)' : string
            elif get_year date_time < 1000
            then $'\'{dt.month}-{dt.day}-{dt.year} {dt:%I}:{dt:%M}:{dt:%S} {dt:%p}\'.format(dt=!date_time)' : string
            else $'\'{dt.month}-{dt.day}-{dt:%y} {dt:%I}:{dt:%M}:{dt:%S} {dt:%p}\'.format(dt=!date_time)' : string
    }

### format_iso8601

In [ ]:
inl format_iso8601 (date_time : date_time) : string =
    backend_switch {
        Fsharp = fun () => date_time |> format "yyyy-MM-ddTHH-mm-ss.fff" : string
        Python = fun () => date_time |> format "%Y-%m-%dT%H-%M-%S.%f" : string
    }

### min_value

In [ ]:
inl min_value () : date_time =
    backend_switch {
        Fsharp = fun () => $'System.DateTime.MinValue' : date_time
        Python = fun () => $'datetime.datetime.min' : date_time
    }

In [ ]:
//// test
///! fsharp
///! cuda

min_value ()
|> format ""
|> _assert_eq "1-1-1 12:00:00 AM"

.py output (Python):
{ name = __assert_eq; expected = 1-1-1 12:00:00 AM }



.fsx output:
{ name = __assert_eq; expected = 1-1-1 12:00:00 AM }


### max_value

In [ ]:
inl max_value () : date_time =
    backend_switch {
        Fsharp = fun () => $'System.DateTime.MaxValue' : date_time
        Python = fun () => $'datetime.datetime.max' : date_time
    }

In [ ]:
//// test
///! fsharp
///! cuda

max_value ()
|> format ""
|> _assert_eq "12-31-99 11:59:59 PM"

.py output (Python):
{ name = __assert_eq; expected = 12-31-99 11:59:59 PM }



.fsx output:
{ name = __assert_eq; expected = 12-31-99 11:59:59 PM }


### to_calendar_utc

In [ ]:
inl to_calendar_utc (timestamp : timestamp) : date_time =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/time/calendar"
            global "import gleam/time/timestamp"
    }
    inl duration = $'calendar.utc_offset' : duration_gleam
    inl (date : date_gleam), (time_of_day : time_of_day_gleam) = $'!timestamp |> timestamp.to_calendar(!duration)'
    (date, time_of_day, duration) |> to

### tempo_date_time

In [ ]:
nominal time_zone_provider =
    `(
        backend_switch `(()) `({}) {
            Gleam = (fun () =>
                global "import tempo"
                        ) : () -> ()
        }
        $'' : $'tempo.TimeZoneProvider'
    )

### unix_epoch

In [ ]:
inl unix_epoch () : date_time =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/time/timestamp"
            inl timestamp = $'timestamp.from_unix_seconds (0)' : timestamp
            timestamp |> to_calendar_utc
        Fsharp = fun () => $'System.DateTime.UnixEpoch' : date_time
        Python = fun () => $'datetime.datetime(1970, 1, 1)' : date_time
    }

In [ ]:
//// test
///! fsharp
///! gleam
///! cuda

unix_epoch ()
|> format ""
|> _assert_eq "1-1-70 12:00:00 AM"

.py output (Python):
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }

.gleam output (Gleam):
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }



.fsx output:
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }


### date_time_milliseconds

In [ ]:
inl date_time_milliseconds
    (year : int) (month : int) (day : int) (hour : int) (minute : int) (second : int) (millisecond : int)
    : date_time
    =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/time/calendar"
            global "import gleam/time/duration"
            global "import gleam/time/timestamp"
            global "import tempo/month"
            inl calendar_month : resultm.result' calendar_month () =
                $"!month |> month.from_int"
            inl calendar_month = calendar_month |> resultm.unbox |> resultm.get
            // TODO: types
            // inl month =
            //     match month with
            //     | 1 => January
            //     | 2 => February
            //     | 3 => March
            //     | 4 => April
            //     | 5 => May
            //     | 6 => June
            //     | 7 => July
            //     | 8 => August
            //     | 9 => September
            //     | 10 => October
            //     | 11 => November
            //     | 12 => December
            // inl month : infer =
            //     match month with
            //     | January => $'calendar.January'
            //     | February => $'calendar.February'
            //     | March => $'calendar.March'
            //     | April => $'calendar.April'
            //     | May => $'calendar.May'
            //     | June => $'calendar.June'
            //     | July => $'calendar.July'
            //     | August => $'calendar.August'
            //     | September => $'calendar.September'
            //     | October => $'calendar.October'
            //     | November => $'calendar.November'
            //     | December => $'calendar.December'
            inl date : date_gleam = $'calendar.Date(#year, #calendar_month, #day)'
            inl nanoseconds = millisecond * 1000000
            inl time_of_day : time_of_day_gleam = $'calendar.TimeOfDay(#hour, #minute, #second, #nanoseconds)'
            inl duration : duration_gleam = $'duration.milliseconds(#millisecond)'
            (date, time_of_day, duration) |> to : date_time
        Fsharp = fun () =>
            $'System.DateTime (!year, !month, !day, !hour, !minute, !second, !millisecond)' : date_time
        Python = fun () =>
            $'datetime.datetime(!year, !month, !day, !hour, !minute, !second, !millisecond)' : date_time
    }

In [ ]:
//// test
///! fsharp
///! gleam
///! cuda

date_time_milliseconds 1970 1 1 0 0 0 0
|> format ""
|> _assert_eq "1-1-70 12:00:00 AM"

.py output (Python):
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }

.gleam output (Gleam):
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }



.fsx output:
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }


### date_time_utc

In [ ]:
inl date_time_utc
    (year : int) (month : int) (day : int) (hour : int) (minute : int) (second : int)
    : date_time
    =
    backend_switch {
        Fsharp = fun () =>
            $'System.DateTime (!year, !month, !day, !hour, !minute, !second, System.DateTimeKind.Utc)' : date_time
        Python = fun () =>
            $'datetime.datetime(!year, !month, !day, !hour, !minute, !second, tzinfo=datetime.timezone.utc)' : date_time
    }

In [ ]:
//// test
///! fsharp
///! cuda

date_time_utc 1970 1 1 0 0 0
|> format ""
|> _assert_eq "1-1-70 12:00:00 AM"

.py output (Python):
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }



.fsx output:
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }


### date_time_kind

In [ ]:
union date_time_kind =
    | Unspecified
    | Utc
    | Local

### specify_date_kind

In [ ]:
inl specify_date_kind (kind : date_time_kind) (date_time : date_time) : date_time =
    backend_switch {
        Fsharp = fun () =>
            inl kind : $'System.DateTimeKind' =
                match kind with
                | Unspecified => $'System.DateTimeKind.Unspecified'
                | Utc => $'System.DateTimeKind.Utc'
                | Local => $'System.DateTimeKind.Local'
            $'System.DateTime.SpecifyKind (!date_time, !kind)' : date_time
        Python = fun () => $'!date_time.replace(tzinfo=None)' : date_time
    }

### to_universal_time

In [ ]:
inl to_universal_time (date_time : date_time) : date_time =
    backend_switch {
        Fsharp = fun () => date_time |> $'_.ToUniversalTime()' : date_time
        Python = fun () => $'!date_time.replace(tzinfo=datetime.timezone.utc)' : date_time
    }

In [ ]:
//// test
///! fsharp
///! cuda

date_time_milliseconds 1970 1 1 0 0 0 0
|> specify_date_kind Utc
|> to_universal_time
|> format ""
|> _assert_eq "1-1-70 12:00:00 AM"

.py output (Python):
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }



.fsx output:
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }


In [ ]:
//// test
///! fsharp
///! cuda

date_time_utc 1970 1 1 0 0 0
|> specify_date_kind Utc
|> format ""
|> _assert_eq "1-1-70 12:00:00 AM"

.py output (Python):
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }



.fsx output:
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }


In [ ]:
//// test
///! fsharp
///! cuda

date_time_utc 1970 1 1 0 0 0
|> specify_date_kind Local
|> format ""
|> _assert_eq "1-1-70 12:00:00 AM"

.py output (Python):
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }



.fsx output:
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }


In [ ]:
//// test
///! fsharp
///! cuda

date_time_utc 1970 1 1 0 0 0
|> specify_date_kind Unspecified
|> format ""
|> _assert_eq "1-1-70 12:00:00 AM"

.py output (Python):
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }



.fsx output:
{ name = __assert_eq; expected = 1-1-70 12:00:00 AM }


### time_span

In [ ]:
nominal time_span_python =
    `(
        backend_switch `(()) `({}) {
            Python = (fun () => global "import datetime") : () -> ()
        }
        $'' : $'datetime.timedelta'
    )
type time_span_switch =
    {
        Fsharp : $'System.TimeSpan'
        Python : time_span_python
    }
nominal time_span = $'backend_switch `(time_span_switch)'

inl time_span x : time_span =
    backend_switch {
        Gleam = fun () => x |> convert : time_span
        Fsharp = fun () => x |> convert : time_span
        Python = fun () => $'datetime.timedelta(!x)' : time_span
    }

### new_time_span

In [ ]:
inl new_time_span (a : date_time) (b : date_time) : time_span =
    $'!b - !a '

### total_seconds

In [ ]:
inl total_seconds (time_span : time_span) : f64 =
    backend_switch {
        Gleam = fun () => $'0' : f64
        Fsharp = fun () => time_span |> $'_.TotalSeconds' : f64
        Python = fun () => $'!time_span.total_seconds()' : f64
    }

### days

In [ ]:
inl days (time_span : time_span) : f64 =
    backend_switch {
        Gleam = fun () => $'0' : f64
        Fsharp = fun () => time_span |> $'_.TotalDays' : f64
        Python = fun () => $'!time_span.days' : f64
    }

### ticks

In [ ]:
inl ticks (date_time : date_time) : timestamp =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/time/timestamp"
            // inl date_time (date, time_of_day, duration) = date_time
            inl date : date_gleam = $'!date_time.0'
            inl time_of_day : time_of_day_gleam = $'!date_time.1'
            inl duration : duration_gleam = $'!date_time.2'
            $'timestamp.from_calendar(!date, !time_of_day, !duration)' : timestamp
        Fsharp = fun () =>
            run_target function
                | Rust (Contract) => fun () => null ()
                | _ => fun () => date_time |> $'_.Ticks'
            : timestamp
        Python = fun () =>
            date_time |> new_time_span (min_value ()) |> total_seconds |> ((*) 10000000) |> convert : timestamp
    }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust

unix_epoch ()
|> ticks
|> _assert_eq' (621355968000000000i64 |> convert)

.py output (Python):
{ name = __assert_eq'; expected = 621355968000000000 }

.rs output:
{ name = __assert_eq'; expected = 621355968000000000 }



.fsx output:
{ name = __assert_eq'; expected = 621355968000000000L }


### hours

In [ ]:
inl hours (time_span : time_span) : i32 =
    backend_switch {
        Gleam = fun () => $'0' : i32
        Fsharp = fun () => time_span |> $'_.Hours' : i32
        Python = fun () => $'!time_span.seconds // 3600' : i32
    }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

(join date_time_milliseconds 2002 2 2 20 22 24 26)
|> new_time_span (date_time_milliseconds 2001 1 1 10 11 12 13)
|> hours
|> _assert_eq 10

.py output (Python):
{ name = __assert_eq; expected = 10 }

.rs output:
{ name = __assert_eq; expected = 10 }

.ts output:
{ name = __assert_eq; expected = 10 }

.py output:
{ name = __assert_eq; expected = 10 }



.fsx output:
{ name = __assert_eq; expected = 10 }


### minutes

In [ ]:
inl minutes (time_span : time_span) : i32 =
    backend_switch {
        Gleam = fun () => $'0' : i32
        Fsharp = fun () => time_span |> $'_.Minutes' : i32
        Python = fun () => $'(!time_span.seconds // 60) % 60' : i32
    }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

(join date_time_milliseconds 2002 2 2 20 22 24 26)
|> new_time_span (date_time_milliseconds 2001 1 1 10 11 12 13)
|> minutes
|> _assert_eq 11

.py output (Python):
{ name = __assert_eq; expected = 11 }

.rs output:
{ name = __assert_eq; expected = 11 }

.ts output:
{ name = __assert_eq; expected = 11 }

.py output:
{ name = __assert_eq; expected = 11 }



.fsx output:
{ name = __assert_eq; expected = 11 }


### seconds

In [ ]:
inl seconds (time_span : time_span) : i32 =
    backend_switch {
        Gleam = fun () => $'0' : i32
        Fsharp = fun () => time_span |> $'_.Seconds' : i32
        Python = fun () => $'!time_span.seconds % 60' : i32
    }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

(join date_time_milliseconds 2002 2 2 20 22 24 26)
|> new_time_span (date_time_milliseconds 2001 1 1 10 11 12 13)
|> seconds
|> _assert_eq 12

.py output (Python):
{ name = __assert_eq; expected = 12 }

.rs output:
{ name = __assert_eq; expected = 12 }

.ts output:
{ name = __assert_eq; expected = 12 }

.py output:
{ name = __assert_eq; expected = 12 }



.fsx output:
{ name = __assert_eq; expected = 12 }


### milliseconds

In [ ]:
inl milliseconds (time_span : time_span) : i32 =
    backend_switch {
        Gleam = fun () => $'0' : i32
        Fsharp = fun () => time_span |> $'_.Milliseconds' : i32
        Python = fun () => $'!time_span.microseconds' : i32
    }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

(join date_time_milliseconds 2002 2 2 20 22 24 26)
|> new_time_span (date_time_milliseconds 2001 1 1 10 11 12 13)
|> milliseconds
|> _assert_eq 13

.py output (Python):
{ name = __assert_eq; expected = 13 }

.rs output:
{ name = __assert_eq; expected = 13 }

.ts output:
{ name = __assert_eq; expected = 13 }

.py output:
{ name = __assert_eq; expected = 13 }



.fsx output:
{ name = __assert_eq; expected = 13 }


### time_span_format

In [ ]:
inl time_span_format (format : string) (time_span : time_span) : string =
    run_target_args (fun () => format, time_span) function
        | TypeScript _
        | Python _ => fun format, time_span =>
            $'!time_span.ToString ("c", System.Globalization.CultureInfo.InvariantCulture)'
        | Rust _ => fun format, time_span =>
            open rust.rust_operators
            inl format = format
            !\\((time_span, format), $'"$0.toString($1)"')
        | _ => fun format, time_span =>
            backend_switch {
                Fsharp = fun () => $'!time_span.ToString !format ' : string
                Python = fun () =>
                    inl date = $'!time_span + !(min_value ())' : date_time
                    inl time = $'!date.strftime("%H:%M:%S")' : string
                    inl days = time_span |> days
                    inl result =
                        if days <= 0
                        then time
                        else $days ++# time
                    inl ms = time_span |> milliseconds
                    if ms <= 0
                    then result
                    else
                        inl ms =
                            ms
                            |> sm'.obj_to_string
                            |> sm'.pad_left 3 '0'
                            |> sm'.pad_right 7 '0'
                        result ++# ms
            }

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

(join date_time_milliseconds 2002 2 2 20 22 24 26)
|> new_time_span (date_time_milliseconds 2001 1 1 10 11 12 13)
|> time_span_format "c"
|> _assert_eq (backend_switch {
    Fsharp = fun () => "397.10:11:12.0130000"
    Python = fun () => "39710:11:120130000"
})

.py output (Python):
{ name = __assert_eq; expected = 39710:11:120130000 }

.rs output:
{ name = __assert_eq; expected = 397.10:11:12.0130000 }

.ts output:
{ name = __assert_eq; expected = 397.10:11:12.0130000 }

.py output:
{ name = __assert_eq; expected = 397.10:11:12.0130000 }



.fsx output:
{ name = __assert_eq; expected = 397.10:11:12.0130000 }


In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

(join date_time_milliseconds 2002 2 2 20 22 24 26)
|> new_time_span (date_time_milliseconds 2002 2 2 10 11 12 26)
|> time_span_format "c"
|> _assert_eq "10:11:12"

.py output (Python):
{ name = __assert_eq; expected = 10:11:12 }

.rs output:
{ name = __assert_eq; expected = 10:11:12 }

.ts output:
{ name = __assert_eq; expected = 10:11:12 }

.py output:
{ name = __assert_eq; expected = 10:11:12 }



.fsx output:
{ name = __assert_eq; expected = 10:11:12 }


In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

(join date_time_milliseconds 2002 2 2 20 22 24 999)
|> new_time_span (date_time_milliseconds 2002 2 2 10 11 12 0)
|> time_span_format "c"
|> _assert_eq (backend_switch {
    Fsharp = fun () => "10:11:12.9990000"
    Python = fun () => "10:11:129990000"
})

.py output (Python):
{ name = __assert_eq; expected = 10:11:129990000 }

.rs output:
{ name = __assert_eq; expected = 10:11:12.9990000 }

.ts output:
{ name = __assert_eq; expected = 10:11:12.9990000 }

.py output:
{ name = __assert_eq; expected = 10:11:12.9990000 }



.fsx output:
{ name = __assert_eq; expected = 10:11:12.9990000 }


### time_zone_info

In [ ]:
nominal time_zone_info = $'System.TimeZoneInfo'

### add_days

In [ ]:
inl add_days (days : i32) (date_time : date_time) : date_time =
    $'!date_time.AddDays' days

### to_calendar

In [ ]:
inl to_calendar (timestamp : timestamp) : date_time =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/time/timestamp"
            global "import gleam/time/calendar"
    }
    inl duration = $'calendar.local_offset()' : duration_gleam
    inl (date : date_gleam), (time_of_day : time_of_day_gleam) = $'!timestamp |> timestamp.to_calendar(!duration)'
    (date, time_of_day, duration) |> to

### now

In [ ]:
inl now () : date_time =
    backend_switch {
        Gleam = fun () =>
            inl timestamp = $'timestamp.system_time ()' : timestamp
            timestamp |> to_calendar
        Fsharp = fun () =>
            run_target function
                | Rust (Contract) => fun () => null ()
                | _ => fun () => $'System.DateTime.Now'
            : date_time
        Python = fun () => $'datetime.datetime.now()' : date_time
    }

### utc_now

In [ ]:
inl utc_now () : date_time =
    backend_switch {
        Fsharp = fun () => $'System.DateTime.UtcNow' : date_time
        Python = fun () => $'datetime.datetime.utcnow()' : date_time
    }

### stopwatch

In [ ]:
nominal stopwatch_python =
    `(
        global "import timeit"
        $'' : $'timeit.default_timer'
    )
type stopwatch_switch =
    {
        Fsharp : $'System.Diagnostics.Stopwatch'
        Python : stopwatch_python
    }
nominal stopwatch = $'backend_switch `(stopwatch_switch)'

inl stopwatch () : stopwatch =
    backend_switch {
        Fsharp = fun () => () |> convert : stopwatch
        Python = fun () => $'`stopwatch ' : stopwatch
    }

In [ ]:
inl stopwatch_elapsed_milliseconds (stopwatch : stopwatch) : i64 =
    $'!stopwatch.ElapsedMilliseconds'

In [ ]:
inl stopwatch_start (stopwatch : stopwatch) : () =
    $'!stopwatch.Start' ()

## rust

### duration'

In [ ]:
nominal duration' =
    `(
        backend_switch `(()) `({}) {
            Fsharp = (fun () => global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::time::Duration\")>]\n#endif\ntype std_time_Duration = class end") : () -> ()
        }
        $'' : $'std_time_Duration'
    )

### date_time'

In [ ]:
nominal date_time' t =
    `(
        backend_switch `(()) `({}) {
            Fsharp = (fun () => global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"chrono::DateTime<$0>\")>]\n#endif\ntype chrono_DateTime<'T> = class end") : () -> ()
        }
        $'' : $'chrono_DateTime<`t>'
    )

### local

In [ ]:
nominal local =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"chrono::Local\")>]\n#endif\ntype chrono_Local = class end"
        $'' : $'chrono_Local'
    )

### naive_date_time

In [ ]:
nominal naive_date_time =
    `(
        backend_switch `(()) `({}) {
            Fsharp = (fun () => global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"chrono::NaiveDateTime\")>]\n#endif\ntype chrono_NaiveDateTime = class end") : () -> ()
        }
        $'' : $'chrono_NaiveDateTime'
    )

## utc

In [ ]:
nominal utc =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"chrono::Utc\")>]\n#endif\ntype chrono_Utc = class end"
        $'' : $'chrono_Utc'
    )

### naive_utc

In [ ]:
inl naive_utc (date_time : date_time' utc) : naive_date_time =
    !\\(date_time, $'"$0.naive_utc()"')

### to_local

In [ ]:
inl to_local (date_time : date_time' utc) : date_time' local =
    inl naive_date_time = date_time |> naive_utc
    !\\(naive_date_time, $'"chrono::offset::TimeZone::from_utc_datetime(&chrono::Local, &$0)"')

### from_timestamp_micros

In [ ]:
inl from_timestamp_micros forall t {number; int}. (timestamp : t) : option (date_time' utc) =
    inl result : optionm'.option' (date_time' utc) =
        !\\(timestamp, $'"chrono::DateTime::from_timestamp_micros($0)"')
    result |> optionm'.unbox

### format'

In [ ]:
inl format' (format : string) (date_time : date_time' utc) : sm'.std_string =
    !\\((date_time, #format), $'"$0.format($1).to_string()"')

### format''

In [ ]:
inl format'' (format : string) (date_time : date_time' _) : sm'.std_string =
    !\\((date_time, #format), $'"$0.format($1).to_string()"')

### format_timestamp

In [ ]:
inl format_timestamp forall t {number; int}. (timestamp : t) =
    inl timestamp = join timestamp
    (timestamp / 1000)
    |> from_timestamp_micros
    |> optionm.map fun x =>
        x
        |> to_local
        |> format'' "%Y-%m-%d %H:%M:%S"
        |> sm'.from_std_string
    |> resultm.from_option

### duration_from_millis

In [ ]:
inl duration_from_millis (ms : u64) : duration' =
    inl ms = join ms
    !\($'"std::time::Duration::from_millis(!ms)"')

## date_time

### time_zone_local

In [ ]:
inl time_zone_local () : time_zone_info =
    run_target function
        | Fsharp _ => fun () =>
            $'System.TimeZoneInfo.Local'
        | Rust (Native) => fun () =>
            open rust.rust_operators
            !\($'"std::sync::Arc::new(chrono::FixedOffset::local_minus_utc(chrono::Local::now().offset()) as i64)"')
        | _ => fun () => null ()

### get_utc_offset

In [ ]:
inl get_utc_offset (time_zone_info : time_zone_info) (date_time : date_time) : time_span =
    run_target function
        | Fsharp _ => fun () => date_time |> $'_.GetUtcOffset' (time_zone_local ())
        | Rust (Native | Wasm) => fun () =>
            open rust.rust_operators
            inl ticks = date_time |> ticks
            // inl ticks = ticks |> rust.rust.emit
            (!\\((date_time, ticks), $'"chrono::FixedOffset::local_minus_utc(&chrono::DateTime::timezone(&chrono::DateTime::fixed_offset(&chrono::DateTime::from_timestamp_nanos($1))))"') : i32)
            |> convert
        | target => fun () =>
            backend_switch {
                Fsharp = fun () => failwith $'$"date_time.get_utc_offset / target: {!target}"' : time_span
                Python = fun () => $'!date_time.utcoffset()' : time_span
            }

### date_time_guid_from_date_time

In [ ]:
let date_time_guid_from_date_time (guid' : guid.guid) (date_time : date_time) =
    inl create prefix time_zone : date_time_guid =
        inl guid_range =
            guid'
            |> sm'.obj_to_string
            |> sm'.range
                (am'.Start ((prefix |> sm'.length |> fun x => x : i32) + (time_zone |> sm'.length)))
                (am'.End eval)
        ($'$"{!prefix}{!time_zone}{!guid_range}"' : string) |> guid.new_guid
    run_target function
        | Rust (Contract) => fun () => null ()
        | Rust (Native | Wasm) => fun () =>
            inl epoch =
                unix_epoch ()
                |> to_universal_time
            inl date_time =
                date_time
                |> specify_date_kind Local
                |> to_universal_time
            inl unixticks =
                match date_time |> ticks, epoch |> ticks with
                | timestamp date_time, timestamp epoch => convert date_time - convert epoch : i64
            inl prefix =
                unixticks / 10
                |> from_timestamp_micros
                |> optionm.map (
                    to_local
                    >> format'' "%Y%m%d-%H%M-%S%f"
                    >> sm'.from_std_string
                    >> fun s => $'$"{!s.[0..17]}-{!s.[18..21]}-{!s.[22]}"'
                )
                |> optionm'.default_value ""
            inl time_zone = date_time |> get_utc_offset (time_zone_local ())
            inl time_zone_signal = if hours time_zone > 0 then 1u8 else 0
            inl time_zone_value = time_zone |> time_span_format (join "hh:mm")
            inl time_zone = $'$"{!time_zone_signal}{!time_zone_value.[0..1]}{!time_zone_value.[3..4]}"'
            create prefix time_zone
        | target => fun () =>
            inl prefix = date_time |> format (join "yyyyMMdd-HHmm-ssff-ffff-f")
            inl time_zone = date_time |> get_utc_offset (time_zone_local ())
            inl time_zone_signal = if hours time_zone > 0 then 1u8 else 0
            inl time_zone_value = time_zone |> time_span_format (join "hhmm")
            inl time_zone = $'$"{!time_zone_signal}{!time_zone_value}"'
            create prefix time_zone

In [ ]:
//// test
///! fsharp
////! cuda
////! rust -d chrono // Ambiguous local time, ranging from 1970-01-01T00:00:00-03:00 to 1970-01-01T00:00:00-04:00

inl suffix = test_guid () |> sm'.obj_to_string |> sm'.range (am'.End fun x => x () - 6i32) (am'.End eval)
unix_epoch ()
|> specify_date_kind Utc
|> to_universal_time
|> date_time_guid_from_date_time (test_guid ())
|> sm'.obj_to_string
|> fun s => s |> _assert_eq' $'$"{!(s |> sm'.slice 0i32 29)}{!suffix}"'

{ name = __assert_eq'; expected = 19700101-0000-0000-0000-000000cba987 }


In [ ]:
//// test
///! fsharp
///! rust -d chrono

inl suffix = test_guid () |> sm'.obj_to_string |> sm'.range (am'.End fun x => x () - 6i32) (am'.End eval)
min_value ()
|> specify_date_kind Local
|> date_time_guid_from_date_time (test_guid ())
|> sm'.obj_to_string
|> fun s => s |> _assert_eq' $'$"00010101-0000-0000-0000-0{!(s |> sm'.slice 25i32 29)}{!suffix}"'

.rs output (rust -d chrono):
{ name = __assert_eq'; expected = 00010101-0000-0000-0000-000000cba987 }



.fsx output:
{ name = __assert_eq'; expected = 00010101-0000-0000-0000-000000cba987 }


In [ ]:
//// test
///! fsharp

inl suffix = test_guid () |> sm'.obj_to_string |> sm'.range (am'.End fun x => x () - 6i32) (am'.End eval)
max_value ()
|> specify_date_kind Utc
|> add_days -1
|> date_time_guid_from_date_time (test_guid ())
|> sm'.obj_to_string
|> fun s => s |> _assert_eq $'$"99991230-2359-5999-9999-9{!(s |> sm'.slice 25i32 29)}{!suffix}"'

{ name = __assert_eq; expected = 99991230-2359-5999-9999-900000cba987 }


In [ ]:
//// test
///! rust -d chrono

inl suffix = test_guid () |> sm'.obj_to_string |> sm'.range (am'.End fun x => x () - 6i32) (am'.End eval)
max_value ()
|> specify_date_kind Utc
|> add_days -1
|> date_time_guid_from_date_time (test_guid ())
|> sm'.obj_to_string
|> fun s => s |> _assert_eq $'$"99991230-2359-5999-9999-0{!(s |> sm'.slice 25i32 29)}{!suffix}"'

{ name = __assert_eq; expected = 99991230-2359-5999-9999-000000cba987 }


In [ ]:
//// test
///! fsharp
///! rust -d chrono

inl suffix = test_guid () |> sm'.obj_to_string |> sm'.range (am'.End fun x => x () - 6i32) (am'.End eval)
unix_epoch ()
|> specify_date_kind Utc
|> add_days 1
|> date_time_guid_from_date_time (test_guid ())
|> sm'.obj_to_string
|> fun s => s |> _assert_eq $'$"19700102-0000-0000-0000-0{!(s |> sm'.slice 25i32 29)}{!suffix}"'

.rs output (rust -d chrono):
{ name = __assert_eq; expected = 19700102-0000-0000-0000-000000cba987 }



.fsx output:
{ name = __assert_eq; expected = 19700102-0000-0000-0000-000000cba987 }


### date_time_from_guid

In [ ]:
inl date_time_from_guid (date_time_guid : date_time_guid) =
    inl date_time_guid = date_time_guid |> sm'.obj_to_string
    inl sm_replace = sm'.replace "-" ""
    run_target_args (fun () => sm_replace) function
        | (Rust _ | TypeScript _) => fun sm_replace =>
            $'System.DateTime.Parse (!date_time_guid.[..24] |> !sm_replace)' : date_time
        | _ => fun sm_replace => $'System.DateTime.ParseExact (!date_time_guid.[..24] |> !sm_replace, "yyyyMMddHHmmssfffffff", null)' : date_time

In [ ]:
//// test

date_time_from_guid (guid.new_guid "00010101-0000-0000-0000-0a9876543210")
|> _assert_eq' (min_value ())

{ name = __assert_eq'; expected = 01/01/0001 00:00:00 }


In [ ]:
//// test

date_time_from_guid (guid.new_guid $'$"99991231-2359-5999-9999-9{(!test_guid () |> string).[^10..]}"')
|> _assert_eq' (max_value ())

{ name = __assert_eq'; expected = 12/31/9999 23:59:59 }


In [ ]:
//// test

date_time_from_guid (guid.new_guid $'$"19700101-0000-0000-0000-0{(!test_guid () |> string).[^10..]}"')
|> _assert_eq' $'System.DateTime.UnixEpoch'

{ name = __assert_eq'; expected = 01/01/1970 00:00:00 }


### timestamp_guid_from_timestamp

In [ ]:
inl timestamp_guid_from_timestamp (guid : guid.guid) (timestamp : timestamp) : timestamp_guid =
    inl guid = guid |> sm'.obj_to_string
    inl timestamp = timestamp |> sm'.obj_to_string |> sm'.pad_left 18i32 '0'
    $'`timestamp_guid $"{!timestamp.[0..7]}-{!timestamp.[8..11]}-{!timestamp.[12..15]}-{!timestamp.[16..17]}{!guid.[21..]}"'

In [ ]:
//// test

timestamp_guid_from_timestamp (test_guid ()) (0i64 |> convert |> timestamp)
|> _assert_eq' (guid.new_guid "00000000-0000-0000-0043-210fedcba987")

{ name = __assert_eq'; expected = 00000000-0000-0000-0043-210fedcba987 }


In [ ]:
//// test

timestamp_guid_from_timestamp (test_guid ()) (999999999999999999i64 |> convert |> timestamp)
|> _assert_eq' (guid.new_guid $'$"99999999-9999-9999-9943-2{(!test_guid () |> string).[^10..]}"')

{ name = __assert_eq'; expected = 99999999-9999-9999-9943-210fedcba987 }


### timestamp_from_guid

In [ ]:
inl timestamp_from_guid (guid : date_time_guid) : timestamp =
    inl guid = guid |> sm'.obj_to_string
    $'`i64 $"{!guid.[0..7]}{!guid.[9..12]}{!guid.[14..17]}{!guid.[19..20]}"'

In [ ]:
//// test

timestamp_from_guid (guid.new_guid "00000000-0000-0000-00dc-ba9876543210")
|> _assert_eq' (0i64 |> convert |> timestamp)

{ name = __assert_eq'; expected = 0L }


In [ ]:
//// test

timestamp_from_guid (guid.new_guid $'$"99999999-9999-9999-99{(!test_guid () |> string).[^14..]}"')
|> _assert_eq' (999999999999999999i64 |> convert |> timestamp)

{ name = __assert_eq'; expected = 999999999999999999L }


### new_guid_from_date_time

In [ ]:
inl new_guid_from_date_time (date_time : date_time) =
    inl guid = guid.new_raw_guid ()
    date_time |> date_time_guid_from_date_time guid

In [ ]:
//// test

utc_now ()
|> new_guid_from_date_time
|> date_time_from_guid
|> fun date_time => new_time_span date_time (utc_now ()) |> total_seconds |> i32
|> _assert_eq 0

{ name = __assert_eq; expected = 0 }


### new_guid_from_timestamp

In [ ]:
inl new_guid_from_timestamp (timestamp : timestamp) =
    inl guid = guid.new_raw_guid ()
    timestamp |> timestamp_guid_from_timestamp guid

In [ ]:
//// test

utc_now ()
|> ticks
|> new_guid_from_timestamp
|> timestamp_from_guid
|> fun (timestamp t) => (convert t - (utc_now () |> ticks |> fun (timestamp x) => convert x)) / 100000i64
|> _assert_eq 0i64

{ name = __assert_eq; expected = 0 }


## main

In [ ]:
inl main () =
    $'let date_time_guid_from_date_time x = !date_time_guid_from_date_time x' : ()
    $'let date_time_from_guid x = !date_time_from_guid x' : ()
    $'let timestamp_guid_from_timestamp x = !timestamp_guid_from_timestamp x' : ()
    $'let timestamp_from_guid x = !timestamp_from_guid x' : ()
    $'let new_guid_from_date_time x = !new_guid_from_date_time x' : ()
    $'let new_guid_from_timestamp x = !new_guid_from_timestamp x' : ()
    $'let format x = !format x' : ()
    $'let format_iso8601 x = !format_iso8601 x' : ()